In [ ]:
%matplotlib inline
from clawpack import pyclaw
import numpy as np
import matplotlib.pyplot as plt
from numpy import sqrt, exp, cos

PyClaw makes use of a small number of essential data structures, including

- `Controller`
- `Solution`
- `State`
- `Solver`
- Geometric objects: `Domain`, `Grid`, `Dimension`, `Patch`

The relationship between some of these objects is shown below.

![image](./pyclaw_solution_structure.png)

Typically, the Solution object itself belongs to a `Controller` object and is paired with a `Solver`, which
is used to advance the solution in time:

<img src="controller.png" alt="drawing" width="450"/>

As shown here, the `Controller` is also responsible for setting up the output from a simulation.

In this notebook we describe in more detail the classes: `Solution`, `State`, `Solver`, and `Controller`.  The geometry
objects are explained in a separate notebook

# `Solver`

The `Solver` contains information about how a `Solution` should evolve in time, such as:

- which basic discretization method to use (i.e. the Lax-Wendroff-LeVeque approach or method of lines)
- which Riemann solver to use
- limiters to use
- time-stepping method

There are separate `Solver` subclasses depending on the number of spatial dimensions (1-3) and
the choice of basic discretization.  Each `Solver` must be initialized with a Riemann solver as
its only argument; the Riemann solver also defines the system of equtions to be solved.  Here we'll
solve the 1D acoustics equations in a homogeneous medium.

In [ ]:
from clawpack import riemann
riemann_solver = riemann.acoustics_1D
solver = pyclaw.ClawSolver1D(riemann_solver)

The `Solver` includes a large number of modifiable parameters, relating to the choice of numerical algorithm.
For instance, one can set a target CFL number and a step-rejection threshold:

In [ ]:
print(solver.cfl_desired)
print(solver.cfl_max)

Many Clawpack Riemann solvers require additional parameters to be set (coefficients of the PDE, properties of the
grid mapping, etc.)  You can find any scalar parameters that must be set as follows:

In [ ]:
for val in dir(riemann_solver.cparam):
    if val[0] != '_':
        print(val)

Because these values are associated with a particular `State`,  will need to set them later when working with the `State`.

Finally, the `Solver` is responsible for the boundary conditions.  There are a few built-in boundary condition types in PyClaw:

- periodic
- reflecting (wall)
- outflow (extrapolation)

For other conditions, a custom boundary condition must be set.

In [ ]:
solver.bc_lower[0] = pyclaw.BC.periodic
solver.bc_upper[0] = pyclaw.BC.periodic

# `State`

A `State` describes the values of a set of fields over some Domain.  It is usually initialized from a `Domain`
and an integer `m` denoting the number of fields (i.e., the number of conserved quantities in the hyperbolic
system of interest).

In [ ]:
x = pyclaw.Dimension(0.0, 1.0, 100, name='x')
domain = pyclaw.Domain(x)
num_eqn = 2
state = pyclaw.State(domain, num_eqn)

A `State` can also optionally hold:

- A set of *auxiliary* fields (`state.aux`) that take values over the domain.  Usually these are spatially-varying coefficients of the PDE, or parameters related to a mapped grid
- A set of scalar values (`state.problem_data`) stored as a Python dictionary.  These may be space-independent coefficients of the PDE, such as the force of gravity or the ratio of specific heats.

Now we can set the physical parameters required by the Riemann solver, in `state.problem_data`:

In [ ]:
rho = 1.0   # Material density
bulk = 1.0  # Material bulk modulus

state.problem_data['rho'] = rho
state.problem_data['bulk'] = bulk
state.problem_data['zz'] = sqrt(rho*bulk)   # Impedance
state.problem_data['cc'] = sqrt(bulk/rho)   # Sound speed

How do we know what parameters need to be set, and what their names should be?  This is explained below in
the section on the `Solver`.

In [ ]:
state.q.shape

It should be noted that `state.q` is created as just a pointer to memory, so its values should
always be set to something meaningful before it is used.

In [ ]:
gamma = 0.; x0 = 0.75
xc = domain.grid.x.centers

state.q[0, :] = exp(-100 * (xc- x0)**2) * cos(gamma * (xc - x0))
state.q[1, :] = 0.0

# `Solution`

The `Solution` object describes some number of fields (usually conserved quantities like mass, momentum, etc.)
that take values over a physical space.  The set of fields is defined by one or more `States` and the physical
space is described by a `Domain`.  Thus, a `Solution` is typically initialized using a list of `States` (often just one) and a `Domain`.
The list of states can also be replaced by just an integer `m`, in which case there will be a single `State` occupying the whole domain,
with `m` fields.

In [ ]:
sol = pyclaw.Solution(state, domain)

In [ ]:
print(sol.domain.grid)

For convenience, if there is only one state then some properties of the state are accessible directly from
the `Solution`:

In [ ]:
sol.state.q.shape

In [ ]:
sol.q.shape

The `Solution` object is not only used to start a simulation, but is also the output data format used by
Clawpack.

# `Controller`

The `Controller` object is technically just a convenience but is virtually always used 
in practice to handle simulations.  It can be initialized without any arguments, but in
order to function properly it requires a `Solution` and a `Solver`.  The `Controller` then
handles the application of the `Solver` to the `Solution`, as well as output.

In [ ]:
claw = pyclaw.Controller()
claw.solution = sol
claw.solver = solver

In [ ]:
print(claw)

There are a large number of properties here, mostly related to output.  The most important are:

- `outdir`: The directory where output files will be written
- `keep_copy`: If true, output will be attached to the `Controller` at the end of the run as a list of `Solutions` in `claw.frames`
- `num_output_times`: By default, output will be written at this number of evenly spaced times
- `tfinal`: the time at which to end the simulation

Setting `keep_copy = True` is very useful when working with relatively small simulations, since you avoid
the need to read/write data from disk.  It is a very bad idea for large simulations, where you may run
out of memory by trying to store so many snapshots of the solution at once.

In [ ]:
claw.tfinal = 1.0
claw.keep_copy = True
claw.output_format = None  # This disables output to disk

In [ ]:
claw.run()

By default, we get a message on the screen each time an output is written.  We also get
a dictionary of diagnostics (accessible via `claw.status`) telling us how many time steps
were taken and how big they were.

# Output

In this example we're working with output in memory, which is stored in `claw.frames`:

In [ ]:
len(claw.frames)

There are 11 output frames because we always get output at $t=0$ in addition to the number of requested outputs.

Each `frame` is a `Solution` object, and the values of the PDE solution can be accessed via `frame.q`:

In [ ]:
plt.plot(xc,claw.frames[0].q[0,:]);

It's very easy to set up an interactive plot where we can look through all the frames:

In [ ]:
from ipywidgets import interact

def plot_frame(i=0):
    plt.plot(xc,claw.frames[i].q[0,:]);

interact(plot_frame,i=(0,10,1));

Or we can get a little fancier:

In [ ]:
pmax = max([np.max(claw.frames[i].q[0,:]) for i in range(11)])
pmin = min([np.min(claw.frames[i].q[0,:]) for i in range(11)])
dp = pmax - pmin

umax = max([np.max(claw.frames[i].q[1,:]) for i in range(11)])
umin = min([np.min(claw.frames[i].q[1,:]) for i in range(11)])
du = umax - umin

def plot_frame(i=0):
    fig, axes = plt.subplots(2,1)
    axes[0].set_xlim(0,1)
    axes[1].set_xlim(0,1)
    axes[0].set_ylim(pmin-dp/10,pmax+dp/10)
    axes[1].set_ylim(umin-du/10,umax+du/10)
    axes[0].plot(xc,claw.frames[i].q[0,:]);
    axes[1].plot(xc,claw.frames[i].q[1,:]);

interact(plot_frame,i=(0,10,1));